In [13]:
# Imports
import os
import gradio as gr
from typing import cast
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
from pydantic import BaseModel

# Markdown output display
from IPython.display import Markdown, display

In [14]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [15]:
# Load Environment Variables
load_dotenv(override=True)

# Ollama Environment Variables
ollama_api_key = env_to_str('OLLAMA_API_KEY')
ollama_base_url = env_to_str('OLLAMA_BASE_URL')
model_mistral_nemo = env_to_str('MODEL_MISTRAL_NEMO')

# Ollama Client
ollama = OpenAI(
    api_key=ollama_api_key,
    base_url=ollama_base_url
)

In [16]:
# Reading the PDF File
reader = PdfReader("./03-lab/linkedin.pdf")
linkedin = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [ ]:
# Displaying the LinkedIn PDF in text format
display(Markdown(linkedin))

In [ ]:
# Write Content to Text File
with open("./03-lab/summary.txt", "w", encoding="utf-8") as file:
    summary = file.write(linkedin)

# Read Content from Text File
with open("./03-lab/summary.txt", "r", encoding="utf-8") as file:
    summary = file.read()

In [ ]:
# Display the Summary
display(Markdown(summary))

In [21]:
# Variables and Prompts
name = "Luis Santiago"

system_prompt = f"""You are acting as {name}. You are answering questions on {name}'s website, 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. 
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. 
Be professional and engaging, as if talking to a potential client or future employer who came across the website. 
If you don't know the answer, say so.
"""

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
# Display the System Prompt
display(Markdown(system_prompt))

In [ ]:
# Gradio Callback Function
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content